In nodo 5 container of antares: `sudo pip install django-material`

In `/LUSTRE/MADMEX/tasks/2019_tasks/app_label_segments` create next file `.caribbean`:

```
SECRET_KEY=
DEBUG=True
DJANGO_LOG_LEVEL=INFO
DATABASE_NAME=
DATABASE_SCHEMA=public
DATABASE_USER=
DATABASE_PASSWORD=
DATABASE_HOST=
DATABASE_PORT=
ALLOWED_HOSTS=
```

## Setting PYTHONPATH, django-project :

In [23]:
import os
os.chdir('/LUSTRE/MADMEX/tasks/2019_tasks/app_label_segments/caribbean_git')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "caribbean.settings")
import django
django.setup()


test import tables:

In [2]:
from caribbean.models import MadmexCountry

In [3]:
name_admin='ATG'
country, _ = MadmexCountry.objects.get_or_create(name=name_admin)

In [4]:
country

<MadmexCountry: MadmexCountry object (67)>

### Example: ingest a shapefile 

(extracted from antares3-sandbox/notebooks/ingest_admin/ingest_admin.ipynb )

In [5]:
import json
from pprint import pprint
import sys
import unicodedata
from datetime import datetime

import fiona
from shapely.geometry import shape, mapping
from shapely.ops import cascaded_union
from django.contrib.gis.geos import GEOSGeometry
from fiona.crs import to_string, from_epsg
from fiona.transform import transform_geom
from pyproj import Proj

In [6]:
name_admin = 'MEX'
column_name = 'NAME_1'
shapefile = '/LUSTRE/MADMEX/tasks/2019_tasks/MEX_adm_shp/MEX_adm1.shp'

In [7]:
with fiona.open(shapefile,
                encoding='utf-8') as src:
    fc = list(src)
    src_crs = src.crs
    to_string_crs = to_string(src_crs)
    proj_crs = Proj(src.crs)
    if not proj_crs.is_latlong():
        fc_proj = [feature_transform(x,"+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs",to_string_crs) for x in fc]
    else:
        fc_proj = fc

In [8]:
shape_list = [shape(feat['geometry']) for feat in fc_proj]

In [9]:
country_shape = cascaded_union(shape_list)

In [10]:
the_geom=GEOSGeometry(country_shape.wkt)
user_id=17
long_name='Mexico'
added=datetime.now().strftime("%Y-%m-%d %H:%M:%S")
image='https://raw.githubusercontent.com/hjnilsson/country-flags/master/png250px/mx.png'

In [11]:
country, _ = MadmexCountry.objects.get_or_create(name=name_admin, the_geom=the_geom,
                                                 added=added,user_id=user_id, long_name=long_name,
                                                 image=image)

/usr/local/lib/python3.6/dist-packages/django/db/models/fields/__init__.py:1421: RuntimeWarning: DateTimeField MadmexCountry.added received a naive datetime (2019-05-27 21:15:52) while time zone support is active.
  RuntimeWarning)


In [12]:
region_geom = MadmexCountry.objects.get(name=name_admin).the_geom
region_geom


## Ingest Features

In [14]:
from caribbean.models import MadmexCaribesample

In [15]:
confidence=0
st_area = 0
country_id=89
tag_id=127 
tag_editable_id=127 
validated=False
added = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

In [19]:
for k in range(0,5):
    shape = shape_list[k]
    geom = GEOSGeometry(shape.wkt,4326)
    _ = MadmexCaribesample.objects.get_or_create(the_geom=geom,
                                                 added=added,
                                                 confidence=confidence,
                                                 st_area=shape_area,
                                                 country_id=country_id,
                                                 tag_id=tag_id,
                                                 tag_editable_id=tag_editable_id,
                                                 validated=validated)